In [ ]:
import sys 
import glob 
import torch
import pandas as pd
import os 
import numpy as np 
import pickle
import rasterio
import tensorflow as tf

sys.path.append('../models/')
sys.path.append('../methods')
import likelihood as lh
from unet_model import UNet
from tqdm.notebook import tqdm 


## Inference 

This notebook runs inference with the trained UNet model over. We store resutls for each location as a pickle file, containing the matrix of probabilities for each image. 

In [ ]:
model = UNet(4, 2, full_size=True)
model.load_state_dict(torch.load('../local/unet_checkpoint.pth'))

In [ ]:
out_dir = '../local/inference_results'
if not os.path.exists(out_dir)
    os.makedirs(out_dir)

In [ ]:
truth_df = pd.read_csv('../local/truth.csv')
truth_df.head()

In [ ]:
dim = 200 


for i, row in tqdm(truth_df.iterrows()): 
    
    if row.label == 2 or \
        os.path.exists(os.path.join(out_dir, f'{row.location_name}.p')): 
        continue 
    
    # sort images so they're in proper order
    ims = sorted(glob.glob(os.path.join(row.dir, '*.tif')))
    
    img_array = np.empty((len(ims), 4, dim, dim))
    for i, im in enumerate(ims): 
    
        with rasterio.open(im) as src:
            b, g, r, n = src.read()

        stack = (r,g,b,n)
        rgb = np.stack(stack, axis=0)
        np_image = ((rgb/rgb.max())*255).astype(np.uint8)
        img_array[i, :, :, :] = np_image
        
    # Apply model to image array         
    output = model(torch.Tensor(img_array)) 
    output = -output.detach().numpy().reshape(
        output.shape[0], 2,dim,dim
    )[:,1,:,:]
    probs = 1 / (1 + np.exp(output))    
    
    # Save output
    with open(os.path.join(out_dir, f'{row.location_name}.p'), 'wb') as f: 
        pickle.dump(probs, f) 
